In [118]:
import glob
import itertools
import json
import os
import pandas
import scipy
from scipy import stats


In [119]:
path = "TutorialData"
dir_list = os.listdir(path)
 
print("Files and directories in '", path, "' :")
 
# prints all files
print(dir_list)

exp_paths = []

session_map = {}
for filename in dir_list:
    if filename.isdigit():
        exp_paths.append(path + "/" + filename + "/Training/Tutorial/")
        #session_map[filename] = user_id
        files = os.listdir(path + "/" + filename + "/Training/Tutorial/")
        name = files[0]
        index = name.find(".")
        user_id = name[index - 13: index]
        session_map[user_id] = int(filename)

print(exp_paths)

user_group_mapping = {}

for i in range(30, 87):
    user_group_mapping[i] = "A"
    if(i % 2 != 0):
        user_group_mapping[i] = "B"

    #special cases
    if(i == 79):
        user_group_mapping[i] = "A"

print(user_group_mapping)

Files and directories in ' TutorialData ' :
['61', '59', '68', '57', '32', '35', '69', '56', '51', '58', '60', '34', '33', '.DS_Store', '45', '86', '44', '43', '38', '36', '31', '65', '54', '53', '30', '37', '39', '55', '63', '64', '46', '79', '41', '83', '48', '70', '84', '71', '82', '49', '40', '47', '78']
['TutorialData/61/Training/Tutorial/', 'TutorialData/59/Training/Tutorial/', 'TutorialData/68/Training/Tutorial/', 'TutorialData/57/Training/Tutorial/', 'TutorialData/32/Training/Tutorial/', 'TutorialData/35/Training/Tutorial/', 'TutorialData/69/Training/Tutorial/', 'TutorialData/56/Training/Tutorial/', 'TutorialData/51/Training/Tutorial/', 'TutorialData/58/Training/Tutorial/', 'TutorialData/60/Training/Tutorial/', 'TutorialData/34/Training/Tutorial/', 'TutorialData/33/Training/Tutorial/', 'TutorialData/45/Training/Tutorial/', 'TutorialData/86/Training/Tutorial/', 'TutorialData/44/Training/Tutorial/', 'TutorialData/43/Training/Tutorial/', 'TutorialData/38/Training/Tutorial/', 'Tuto

In [120]:
def parseFileNumber(fileName):
    fragments = fileName.split('_')
    return float(fragments[1])

In [121]:
responsiveData = []
experimentEvents = []

def parseFrame(frames, user_id):
    frames_list = []
    for frame in frames:
        frameData = {
            "userID": float,
            "frameNum": int,
            "unixTime": float,
            "timestamp": float,
            "hPos": {"x":float,"y":float,"z":float},
            "hDir": {"x":float,"y":float,"z":float},
            "hRot": {"i":float,"j":float,"k":float},
            "hAngl": {"x":float,"y":float,"z":float},
            "gazeOrigin": {"x":float,"y":float,"z":float},
            "gazeDirection": {"x":float,"y":float,"z":float},
            "rightHandRay": {"x":float,"y":float,"z":float},
            "leftHandRay": {"x":float,"y":float,"z":float},
            "experimentEvents": [],
            "responsiveData": []
        }
        responsiveEvent = frame["responsiveData"]
        experimentEvent = frame["experimentEvents"]
        frameData["userID"] = user_id
        frameData["frameNum"] = frame["frameNum"]
        frameData["unixTime"] = frame["unixTime"]
        frameData["timestamp"] = frame["timestamp"]
        frameData["hPos"] = frame["hPos"]
        frameData["hDir"] = frame["hDir"]
        frameData["hRot"] = frame["hRot"]
        frameData["hAngl"] = frame["hAngl"]
        frameData["gazeOrigin"] = frame["gazeOrigin"]
        frameData["gazeDirection"] = frame["gazeDirection"]
        frameData["rightHandRay"] = frame["rightHandRay"]
        frameData["leftHandRay"] = frame["leftHandRay"]
        frameData["experimentEvents"] = experimentEvent
        frameData["responsiveData"] = responsiveEvent
        frames_list.append(frameData)
        if responsiveEvent:
            responsiveData.append(responsiveEvent)
        if experimentEvent:
            experimentEvents.append(experimentEvent)
    return frames_list

def parseSession(session, user_id):
    sessionData = {
    "userID": float,
    "numFrames": int,
    "task": str,
    "start_time": float,
    "end_time": float,
    "total_time": float,
    "sessionNumber": int,
    "isResponsive": int,
    "frames": []
    }
    sessionData["userID"] = user_id
    sessionData["numFrames"] = session["numFrames"]
    sessionData["task"] = session["task"]
    sessionData["start_time"] = session["start_time"]
    sessionData["end_time"] = session["end_time"]
    sessionData["total_time"] = session["total_time"]
    sessionData["sessionNumber"] = session["sessionNumber"]
    sessionData["isResponsive"] = session["isResponsive"]
    return sessionData


In [122]:
data = []
frames = []
allsession = []
user_id = 0

for exp in exp_paths:
    experiment_data = {
    "path" : str,
    "sessions": {}
    }
    sessions = {}
    experiment = os.listdir(exp)
    current_path = exp
    experiment_data["path"] = current_path
    experiment.sort(key=parseFileNumber)
    for filename in experiment:
        if filename.endswith(".json"):
            # Prints only text file present in My Folder
            print(filename)
            firstFileFlag = False
            if("Session_0" in filename):
                firstFileFlag = True
            with open(current_path + filename, 'r') as f:
                json_data = json.load(f)
                if(firstFileFlag):
                    user_id = session_map[json_data["obj"]["userID"]]
                    session = parseSession(json_data["obj"]["sessionRecordings"][0], user_id)
                    frame = parseFrame(json_data["obj"]["sessionRecordings"][0]["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
                else:
                    session = parseSession(json_data, user_id)
                    frame = parseFrame(json_data["frames"], user_id)
                    session["frames"] = frame
                    num = session["sessionNumber"]
                    sessions[num] = session
                    frames.append(frame)
                    allsession.append(session)
    experiment_data["sessions"] = sessions
    data.append(experiment_data)

#print(data)


TutorialSession_0_1653321690425.json
TutorialSession_1_1653321690425.json
TutorialSession_2_1653321690425.json
TutorialSession_3_1653321690425.json
TutorialSession_4_1653321690425.json
TutorialSession_5_1653321690425.json
TutorialSession_6_1653321690425.json
TutorialSession_7_1653321690425.json
TutorialSession_8_1653321690425.json
TutorialSession_0_1652906031540.json
TutorialSession_1_1652906031540.json
TutorialSession_2_1652906031540.json
TutorialSession_3_1652906031540.json
TutorialSession_4_1652906031540.json
TutorialSession_5_1652906031540.json
TutorialSession_6_1652906031540.json
TutorialSession_7_1652906031540.json
TutorialSession_8_1652906031540.json
TutorialSession_0_1653068647666.json
TutorialSession_1_1653068647666.json
TutorialSession_2_1653068647666.json
TutorialSession_3_1653068647666.json
TutorialSession_4_1653068647666.json
TutorialSession_5_1653068647666.json
TutorialSession_6_1653068647666.json
TutorialSession_7_1653068647666.json
TutorialSession_8_1653068647666.json
T

In [123]:
allsession

sessionsUnpacked = list(itertools.chain(allsession))
sessionsDf = pandas.DataFrame(sessionsUnpacked)
sessionsDf = sessionsDf.sort_values(by='sessionNumber')

sessionsDf = sessionsDf.loc[sessionsDf["sessionNumber"] < 8 ]
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames
0,61,352,Select a hologram or button within arm's reach...,1653321711306,1653321726336,15030,0,False,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165..."


In [124]:
sessionsDf["group"] = sessionsDf["userID"]

for i in sessionsDf.index:
    sessionsDf["group"][i] = user_group_mapping[sessionsDf["userID"][i]]

<ipython-input-124-65844d124438>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessionsDf["group"][i] = user_group_mapping[sessionsDf["userID"][i]]
/Users/abbywysopal/miniconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [125]:
framesUnpacked = list(itertools.chain(*sessionsDf["frames"]))
framesDf = pandas.DataFrame(framesUnpacked)
print(framesDf.shape)
framesDf["group"] = framesDf["userID"] % 2

for i in framesDf.index:
    framesDf["group"][i] = user_group_mapping[framesDf["userID"][i]]

(70197, 14)


<ipython-input-125-b3ff4c52bcf7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  framesDf["group"][i] = user_group_mapping[framesDf["userID"][i]]


In [126]:
users = framesDf.loc[framesDf["experimentEvents"].str.len() != 0]
print(users.shape)

experimentUnpacked = list(itertools.chain(*users["experimentEvents"]))
experimentDf = pandas.DataFrame(experimentUnpacked)
experimentDf.head(1)
print(experimentDf.shape)
experimentDf.shape

experimentDf["userID"] = users["userID"]

for i in users.index:
    eventDF = pandas.DataFrame(users["experimentEvents"][i])

    for j in eventDF.index:
        time = eventDF["unixTime"][j]
        userID = users["userID"][i]
        mask = experimentDf["unixTime"] == time
        experimentDf.loc[mask, "userID"] = userID

experimentDf.head(1)

(627, 15)
(1292, 10)


,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1653321711236,09-01-51-23,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,61.0


In [127]:
experimentDf

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1653321711236,09-01-51-23,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,61.0
1,1652992192159,13-29-52-15,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,44.0
2,1653501165793,10-52-45-79,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,86.0
3,1653002894334,16-28-14-33,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,45.0
4,1652739175706,15-12-55-70,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,33.0
...,...,...,...,...,...,...,...,...,...,...,...
1287,1652722470709,10-34-30-70,False,end,7,Select Input,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.0, 'y': 0.0, 'z': 0.75}",7,7,31.0
1288,1652722470789,10-34-30-78,False,start,8,Scroll,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.0, 'y': 0.0, 'z': 0.75}",8,,31.0
1289,1652809158842,10-39-18-84,False,entered,7,Select Input,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.0, 'y': 0.0, 'z': 0.75}",7,7,36.0
1290,1652809158842,10-39-18-84,False,end,7,Select Input,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.0, 'y': 0.0, 'z': 0.75}",7,7,36.0


In [128]:
startEvents = experimentDf.loc[experimentDf["eventName"] == "start"]
startEvents.head(3)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
0,1653321711236,09-01-51-23,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,61.0
1,1652992192159,13-29-52-15,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,44.0
2,1653501165793,10-52-45-79,False,start,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,,86.0


In [129]:
sessionsDf

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group
0,61,352,Select a hologram or button within arm's reach...,1653321711306,1653321726336,15030,0,False,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",B
135,44,217,Select a hologram or button within arm's reach...,1652992192229,1652992203978,11749,0,False,"[{'userID': 44, 'frameNum': 1, 'unixTime': 165...",A
126,86,83,Select a hologram or button within arm's reach...,1653501165863,1653501170848,4985,0,False,"[{'userID': 86, 'frameNum': 1, 'unixTime': 165...",A
117,45,253,Select a hologram or button within arm's reach...,1653002894394,1653002908465,14071,0,False,"[{'userID': 45, 'frameNum': 1, 'unixTime': 165...",B
108,33,236,Select a hologram or button within arm's reach...,1652739175795,1652739189896,14101,0,False,"[{'userID': 33, 'frameNum': 1, 'unixTime': 165...",B
...,...,...,...,...,...,...,...,...,...,...
133,86,101,Scroll down by selecting and holding the blue ...,1653501224914,1653501234994,10080,7,False,"[{'userID': 86, 'frameNum': 618, 'unixTime': 1...",A
115,33,244,Scroll down by selecting and holding the blue ...,1652739403439,1652739427960,24521,7,False,"[{'userID': 33, 'frameNum': 2324, 'unixTime': ...",B
304,48,160,Scroll down by selecting and holding the blue ...,1653425001461,1653425033334,31873,7,False,"[{'userID': 48, 'frameNum': 2486, 'unixTime': ...",A
178,31,191,Scroll down by selecting and holding the blue ...,1652722470789,1652722489959,19170,7,False,"[{'userID': 31, 'frameNum': 2078, 'unixTime': ...",B


In [130]:
sessionsDf["Actual_Start_Time"] = sessionsDf["start_time"]

for i in startEvents.index:
    userID = startEvents["userID"][i]
    task_num = startEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_Start_Time"] = startEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time
0,61,352,Select a hologram or button within arm's reach...,1653321711306,1653321726336,15030,0,False,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",B,1653321711306


In [131]:
endEvents = experimentDf.loc[experimentDf["eventName"] == "end"]
endEvents.head(1)

,unixTime,systemTime,isResponsive,eventName,task_number,task_type,object_scale,object_position,correct_answer,guess,userID
42,1652733103060,13-31-43-06,False,end,1,Near Select Example,"{'x': 1.0, 'y': 1.0, 'z': 1.0}","{'x': 0.5910000205039978, 'y': -0.166000008583...",1,1,32.0


In [132]:
sessionsDf["Actual_End_Time"] = sessionsDf["end_time"]

for i in endEvents.index:
    userID = endEvents["userID"][i]
    task_num = endEvents["task_number"][i]
    mask = (sessionsDf["userID"] == userID) & (sessionsDf["sessionNumber"] == task_num)
    sessionsDf.loc[mask, "Actual_End_Time"] = endEvents["unixTime"][i]

sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time,Actual_End_Time
0,61,352,Select a hologram or button within arm's reach...,1653321711306,1653321726336,15030,0,False,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",B,1653321711306,1653321726336


In [133]:
end_time_diff = sessionsDf["end_time"].to_numpy() - sessionsDf["Actual_End_Time"].to_numpy()
end_time_diff

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,  17000,  22610,  16810,  63481,  16570,  50350,
        51471,  44240,  90209,  15830,  35550,  11970, 303706,  70331,
        41920,  61631,  41920,  21780,  26467,   8650,   4645,  29711,
        95533,  24400,  25192,   9081,  38789,  42801,  57171,  19780,
        11150,  17031,  17619,  17811, 107571,  17280,  12070,  26480,
        36041,  45060,  75011,  24900,  13970,  11300,  11060,  18460,
         6909,  54391,  16901,  12971,  10060,   7871,  10560,   2460,
         4650,   1660,  12610,  11584,  41091,  13480,  12580,  11771,
        11160,  13761,   6580,  11430,  11645,   6240,   5870,   6550,
      

In [134]:
start_time_diff = sessionsDf["start_time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
start_time_diff

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,  11020,  14041,   7141,   6780,  14510,  15578,
         9491,    180,  97463,   6351,  14212,  22190,  12550,  16180,
        14231,  14130,  14231,  15630,   9589,   8761,   5135,  19043,
        26193,  10250,  11910,  14520,  38821,  14290,  25521,   7121,
        15079,   8651,  35970, 146472,  13401,   9070,  15210,  47886,
        13970,   7588,  16510,  11880,  42811,  70381,  16990,  61671,
       303976,  17580,  24910,  41930,  21781,  29701,  24400,   8640,
        11140,   4651,  57201,  95556,  26402,   9071,  17031,  38791,
        25211,  12050,  51521,  36066,  50155,  11940,  16820,  19790,
      

In [135]:
sessionsDf["Actual_Total_Time"] = sessionsDf["Actual_End_Time"].to_numpy() - sessionsDf["Actual_Start_Time"].to_numpy()
sessionsDf.head(1)

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time,Actual_End_Time,Actual_Total_Time
0,61,352,Select a hologram or button within arm's reach...,1653321711306,1653321726336,15030,0,False,"[{'userID': 61, 'frameNum': 1, 'unixTime': 165...",B,1653321711306,1653321726336,15030


In [136]:
total_time_diff = sessionsDf["total_time"].to_numpy() - sessionsDf["Actual_Total_Time"].to_numpy()
total_time_diff

array([      0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
             0,       0,       0,       0,       0,       0,       0,
          5980,    8569,    9669,   56701,    2060,   34772,   41980,
         44060,   -7254,    9479,   21338,  -10220,  291156,   54151,
         27689,   47501,   27689,    6150,   16878,    -111,    -490,
         10668,   69340,   14150,   13282,   -5439,     -32,   28511,
         31650,   12659,   -3929,    8380,  -18351, -128661,   94170,
          8210,   -3140,  -21406,   22071,   37472,   58501,   13020,
        -28841,  -59081,   -5930,  -43211, -297067,   36811,   -8009,
        -28959,  -11721,  -21830,  -13840,   -6180,   -6490,   -2991,
        -44591,  -83

In [137]:
#task 0
i = 0
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['total_time'].mean()
taskRStd = task_r['total_time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['total_time'].mean()
taskNStd = task_n['total_time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['total_time'], task_n['total_time']))

responsive task 0 Time: nan ± nan
normal task 0 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [138]:
#Task 1
i = 1
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 1 Time: nan ± nan
normal task 1 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [139]:
#Task 2
i = 2
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 2 Time: nan ± nan
normal task 2 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [140]:
#task 3
i = 3
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 3 Time: nan ± nan
normal task 3 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [141]:
#task 4
i = 4
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 4 Time: nan ± nan
normal task 4 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [142]:
#task 5
i = 5
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 5 Time: nan ± nan
normal task 5 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [143]:
#task 6
i = 6
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 6 Time: nan ± nan
normal task 6 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [144]:
#task 7
i = 7
taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
taskDf
task_r = taskDf.loc[taskDf["group"] == 1]
task_n = taskDf.loc[taskDf["group"] == 0]

taskRAvgTime = task_r['Actual_Total_Time'].mean()
taskRStd = task_r['Actual_Total_Time'].std()
print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

taskNAvgTime = task_n['Actual_Total_Time'].mean()
taskNStd = task_n['Actual_Total_Time'].std()
print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

print(scipy.stats.ttest_ind(task_r['Actual_Total_Time'], task_n['Actual_Total_Time']))

responsive task 7 Time: nan ± nan
normal task 7 Time: nan ± nan
Ttest_indResult(statistic=nan, pvalue=nan)


In [153]:
#each task in a loop
for i in range(0,8):
    print(f"TASK {str(i)}:")
    taskDf = sessionsDf.loc[sessionsDf["sessionNumber"] == i]
    taskDf
    task_r = taskDf.loc[taskDf["group"] == "B"]
    task_n = taskDf.loc[taskDf["group"] == "A"]

    taskRAvgTime = task_r['total_time'].mean()
    taskRStd = task_r['total_time'].std()
    print(f"responsive task {str(i)} Time: {taskRAvgTime:.3f} ± {taskRStd:.3f}")

    taskNAvgTime = task_n['total_time'].mean()
    taskNStd = task_n['total_time'].std()
    print(f"normal task {str(i)} Time: {taskNAvgTime:.3f} ± {taskNStd:.3f}")

    print(scipy.stats.ttest_ind(task_r['total_time'], task_n['total_time']))

TASK 0:
responsive task 0 Time: 21743.143 ± 30079.995
normal task 0 Time: 17809.333 ± 20031.060
Ttest_indResult(statistic=0.4988192012398474, pvalue=0.6206389902049052)
TASK 1:
responsive task 1 Time: 43208.095 ± 62203.370
normal task 1 Time: 40736.429 ± 29918.610
Ttest_indResult(statistic=0.1640953365366234, pvalue=0.8704824774877853)
TASK 2:
responsive task 2 Time: 13810.429 ± 12206.047
normal task 2 Time: 11054.905 ± 4182.853
Ttest_indResult(statistic=0.9786508616835858, pvalue=0.3336346853476828)
TASK 3:
responsive task 3 Time: 10720.524 ± 12176.192
normal task 3 Time: 11077.619 ± 19426.227
Ttest_indResult(statistic=-0.07137569573895082, pvalue=0.9434544368307389)
TASK 4:
responsive task 4 Time: 8840.762 ± 4631.565
normal task 4 Time: 18885.810 ± 39822.608
Ttest_indResult(statistic=-1.1481915009448411, pvalue=0.2577100207241299)
TASK 5:
responsive task 5 Time: 10803.952 ± 8337.546
normal task 5 Time: 12148.571 ± 20345.555
Ttest_indResult(statistic=-0.2802401139779609, pvalue=0.7807

In [146]:
usersDf = sessionsDf[["userID"]]
usersDf = usersDf.drop_duplicates()
usersDf = usersDf.sort_values("userID")
usersDf

,userID
207,30
171,31
36,32
108,33
99,34
45,35
162,36
216,37
153,38
225,39


In [155]:
allTasksDf = usersDf

In [156]:
for i in range(0,8):
    allTasksDf[f"task{i}"] = i

allTasksDf["all_tasks"] = 0
allTasksDf["group"] = allTasksDf["userID"] % 2
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,all_tasks,group
207,30,0,1,2,3,4,5,6,7,0,0
171,31,0,1,2,3,4,5,6,7,0,1
36,32,0,1,2,3,4,5,6,7,0,0
108,33,0,1,2,3,4,5,6,7,0,1
99,34,0,1,2,3,4,5,6,7,0,0
45,35,0,1,2,3,4,5,6,7,0,1
162,36,0,1,2,3,4,5,6,7,0,0
216,37,0,1,2,3,4,5,6,7,0,1
153,38,0,1,2,3,4,5,6,7,0,0
225,39,0,1,2,3,4,5,6,7,0,1


In [157]:
for i in allTasksDf.index:
    user = allTasksDf["userID"][i]
    userSessions = sessionsDf.loc[sessionsDf["userID"] == user]
    total = 0
    task = userSessions.loc[userSessions["sessionNumber"] == 0]
    allTasksDf.loc[i, "task0"] = task["total_time"].to_numpy()[0]
    total = int(task["total_time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 1]
    allTasksDf.loc[i, "task1"] = task["total_time"].to_numpy()[0]
    total += int(task["total_time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 2]
    allTasksDf.loc[i, "task2"] = task["total_time"].to_numpy()[0]
    total += int(task["total_time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 3]
    allTasksDf.loc[i, "task3"] = task["total_time"].to_numpy()[0]
    total += int(task["total_time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 4]
    allTasksDf.loc[i, "task4"] = task["total_time"].to_numpy()[0]
    total += int(task["total_time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 5]
    allTasksDf.loc[i, "task5"] = task["total_time"].to_numpy()[0]
    total += int(task["total_time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 6]
    allTasksDf.loc[i, "task6"] = task["total_time"].to_numpy()[0]
    total += int(task["total_time"].to_numpy()[0])
    task = userSessions.loc[userSessions["sessionNumber"] == 7]
    allTasksDf.loc[i, "task7"] = task["total_time"].to_numpy()[0]
    total += int(task["total_time"].to_numpy()[0])
    allTasksDf.loc[i, "all_tasks"] = total
    if(task["isResponsive"].to_numpy()[0] == True):
        allTasksDf.loc[i, "group"] = "B"

#allTasksDf = allTasksDf.drop(columns=["task{i}"])

In [158]:
allTasksDf = allTasksDf.sort_values("all_tasks")
allTasksDf

,userID,task0,task1,task2,task3,task4,task5,task6,task7,all_tasks,group
279,41,6960,16720,5840,4760,4420,4960,12900,11190,67750,1
126,86,4985,4565,1649,2879,25442,4210,15160,10080,68970,0
288,83,8591,8560,2450,3305,7860,4301,15171,20280,70518,1
36,32,10835,16900,11040,3991,4500,6340,16059,10280,79945,0
198,53,22000,11860,6230,5310,4190,3830,15341,12030,80791,1
45,35,14880,11050,4630,6620,10400,3090,23288,6860,80818,1
90,60,6921,19690,6520,4850,4580,4300,18051,20689,85601,0
369,78,8460,16940,12550,3920,11510,5541,19451,10581,88953,0
135,44,11749,25101,11120,3870,4079,3770,17710,14341,91740,0
63,56,8890,17180,4425,7115,6758,3001,19500,24895,91764,0


In [159]:
groupA = allTasksDf.loc[allTasksDf["group"] == 0]
groupB = allTasksDf.loc[allTasksDf["group"] == 1]

groupATime = groupA['all_tasks'].mean()
groupAStd = groupA['all_tasks'].std()
print(f"group A {str(i)} Time: {groupATime:.3f} ± {groupAStd:.3f}")

groupBTime = groupB['all_tasks'].mean()
groupBStd = groupB['all_tasks'].std()
print(f"group B {str(i)} Time: {groupBTime:.3f} ± {groupBStd:.3f}")

print(scipy.stats.ttest_ind(groupB['all_tasks'], groupA['all_tasks']))

group A 126 Time: 159391.650 ± 106024.572
group B 126 Time: 170211.952 ± 87302.945
Ttest_indResult(statistic=0.35748066878599394, pvalue=0.7226593564409451)


In [160]:
sessionsDf.loc[(sessionsDf["userID"] == 30) & (sessionsDf["sessionNumber"] == 0)]

,userID,numFrames,task,start_time,end_time,total_time,sessionNumber,isResponsive,frames,group,Actual_Start_Time,Actual_End_Time,Actual_Total_Time
207,30,2009,Select a hologram or button within arm's reach...,1652717155289,1652717170466,15177,0,False,"[{'userID': 30, 'frameNum': 1, 'unixTime': 165...",A,1652717155289,1652717170466,15177
